In [296]:
import urllib.request
from bs4 import BeautifulSoup, Comment
import string
import pandas as pd

In [228]:
baseURL = "https://www.basketball-reference.com"
url="https://www.basketball-reference.com/players/"

In [184]:
alphabet=string.ascii_lowercase
playerURL=[]
playerID=[]
playerName=[]
yearStart=[]
yearEnd=[]
position=[]
height=[]
weight=[]
birthday=[]
college=[]

In [185]:
for i in range(0,26):
    playerURL.append(url+alphabet[i])
del playerURL[23]

In [186]:
for i in range(0,25):
    x = urllib.request.urlopen(playerURL[i]).read()
    soup = BeautifulSoup(x, "lxml")
    playerTag=soup.findAll(attrs={"data-stat" : "player"})
    YearMinTag=soup.findAll(attrs={"data-stat" : "year_min"})
    YearMaxTag=soup.findAll(attrs={"data-stat" : "year_max"})
    positionTag=soup.findAll(attrs={"data-stat" : "pos"})
    heightTag=soup.findAll(attrs={"data-stat" : "height"})
    weightTag=soup.findAll(attrs={"data-stat" : "weight"})
    bdayTag=soup.findAll(attrs={"data-stat" : "birth_date"})
    collegeTag=soup.findAll(attrs={"data-stat" : "college_name"})

    for i in range(1, len(playerTag)):
        playerID.append(playerTag[i].attrs["data-append-csv"])
        playerName.append(playerTag[i].a.contents[0])
        yearStart.append(YearMinTag[i].contents[0])
        yearEnd.append(YearMaxTag[i].contents[0])
        if(not positionTag[i].contents):
            position.append("")
        else:
            position.append(positionTag[i].contents[0])
        if(not heightTag[i].contents):
            height.append("")
        else:
            height.append(heightTag[i].contents[0])
        if(not weightTag[i].contents):
            weight.append("")
        else:
            weight.append(weightTag[i].contents[0])
        if(not bdayTag[i].contents):
            birthday.append("")
        else:
            birthday.append(bdayTag[i].a.contents[0])
        if(not collegeTag[i].contents):
            college.append("")
        else:
            college.append(collegeTag[i].a.contents[0])


In [187]:
labels = ['Player_ID', 'Name', 'YrStart', 'YrEnd', 'Pos', 'Height', 'Weight', 'Birth_date', 'College']
player_df = pd.DataFrame({"Player_ID": playerID,
                         "Name": playerName,
                         "YrStart": yearStart,
                         "YrEnd": yearEnd,
                         "Pos": position,
                         "Height": height,
                         "Weight": weight,
                         "Birth_date" : birthday,
                         "College" : college})
player_df=player_df[labels]

#fill NAs?

In [189]:
player_df.to_csv("NBA_Players.csv")

In [192]:
#Box Score Scrape
box_score_URL = "https://www.basketball-reference.com/boxscores/"

In [257]:
def gameBoxScoreLinks(a, b, c):
    #arguments represent date (a->month (MM), b-> day (DD), c-> year(YYYY))
    #if month or day is less than 10, write 5 as "5" not "05"
    #returns links to box score game links
    urlBox=[]
    dateURL = "?month=" + str(a) + "&day=" + str(b) +"&year=" + str(c)
    gameURL = box_score_URL + dateURL
    x = urllib.request.urlopen(gameURL).read()
    gameSoup = BeautifulSoup(x, "lxml")
    gameTag = gameSoup.findAll("td", {"class" : "right gamelink"})
    
    for i in range(0,len(gameTag)):
        urlBox.append(baseURL + gameTag[i].a["href"])
        
    return urlBox

In [258]:
urlBox=gameBoxScoreLinks(2,7,2009)

In [429]:
def gameBoxScore(urlBox):
    #takes a list of urls of box scores
    team1=[]
    team2=[]
    date=[]
    playerID=[]
    minutes=[]
    fg=[]
    fga=[]
    fgpct=[]
    fg3 = []
    fg3a=[]
    fg3pct=[]
    ft=[]
    fta=[]
    ftpct=[]
    orb=[]
    drb=[]
    trb=[]
    ast=[]
    stl=[]
    blk=[]
    tov=[]
    pf=[]
    pts=[]
    plus_minus=[]
    ts_pct=[]
    efg_pct=[]
    off_rtg=[]
    def_rtg=[]
    finalMinutes=[]
    for i in range(0, len(urlBox)):
        a = urllib.request.urlopen(urlBox[i]).read()
        boxSoup = BeautifulSoup(a, "lxml")
        headerTag = boxSoup.findAll("th", {"class": "left"})
        for i in range(0,len(headerTag)):
            if(headerTag[i].has_attr("data-append-csv")):
                if(headerTag[i]["data-append-csv"] not in playerID):
                    playerID.append(headerTag[i]["data-append-csv"])
                    date.append(boxSoup.title.contents[0].split(", ")[1] + boxSoup.title.contents[0].split(",")[2].split(" |")[0])
                    team1.append(boxSoup.title.contents[0].split(" at ")[0])
                    team2.append(boxSoup.title.contents[0].split(" at ")[1].split(" ")[0] + " " + boxSoup.title.contents[0].split(" at ")[1].split(" ")[1])
        minuteTag=boxSoup.findAll("td", {"class": "right ", "data-stat": "mp"})
        for i in range(0,len(minuteTag)):
            if(minuteTag[i].parent.parent.name !="tfoot"):
                if(minuteTag[i].contents not in minutes):
                    minutes.append(minuteTag[i].contents)  
        for i in range(0, len(minutes)):
            finalMinutes.append(minutes[i])
        fgTag=boxSoup.findAll("td", {"class": "right ", "data-stat": "fg"})
        for i in range(0,len(fgTag)):
            if(fgTag[i].parent.parent.name != "tfoot"):
                fg.append(fgTag[i].contents) 
        fgaTag=boxSoup.findAll("td", {"class": "right ", "data-stat": "fga"})
        for i in range(0,len(fgaTag)):
            if(fgaTag[i].parent.parent.name!="tfoot"):
                fga.append(fgaTag[i].contents)  
        fgpctTag=boxSoup.findAll("td", {"class": "right ", "data-stat": "fg_pct"})
        for i in range(0,len(fgpctTag)):
            if(fgpctTag[i].parent.parent.name!="tfoot"):
                fgpct.append(fgpctTag[i].contents)
        fg3Tag=boxSoup.findAll("td", {"class": "right ", "data-stat": "fg3"})
        for i in range(0,len(fg3Tag)):
            if(fg3Tag[i].parent.parent.name!="tfoot"):
                fg3.append(fg3Tag[i].contents)
        fg3aTag=boxSoup.findAll("td", {"class": "right ", "data-stat": "fg3a"})
        for i in range(0,len(fg3aTag)):
            if(fg3aTag[i].parent.parent.name!="tfoot"):
                fg3a.append(fg3aTag[i].contents)
        fg3pctTag=boxSoup.findAll("td", {"class": "right ", "data-stat": "fg3_pct"})
        for i in range(0,len(fg3pctTag)):
            if(fg3pctTag[i].parent.parent.name!="tfoot"):
                fg3pct.append(fg3pctTag[i].contents)
        ftTag=boxSoup.findAll("td", {"class": "right ", "data-stat": "ft"})
        for i in range(0,len(ftTag)):
            if(ftTag[i].parent.parent.name!="tfoot"):
                ft.append(ftTag[i].contents)  
        ftaTag=boxSoup.findAll("td", {"class": "right ", "data-stat": "fta"})
        for i in range(0,len(ftaTag)):
            if(ftaTag[i].parent.parent.name!="tfoot"):
                fta.append(ftaTag[i].contents)  
        ftpctTag=boxSoup.findAll("td", {"class": "right ", "data-stat": "ft_pct"})
        for i in range(0,len(ftpctTag)):
            if(ftpctTag[i].parent.parent.name!="tfoot"):
                ftpct.append(ftpctTag[i].contents) 
        orbTag=boxSoup.findAll("td", {"class": "right ", "data-stat": "orb"})
        for i in range(0,len(orbTag)):
            if(orbTag[i].parent.parent.name!="tfoot"):
                orb.append(orbTag[i].contents)
        drbTag=boxSoup.findAll("td", {"class": "right ", "data-stat": "drb"})
        for i in range(0,len(drbTag)):
            if(drbTag[i].parent.parent.name!="tfoot"):
                drb.append(drbTag[i].contents)
        trbTag=boxSoup.findAll("td", {"class": "right ", "data-stat": "trb"})
        for i in range(0,len(trbTag)):
            if(trbTag[i].parent.parent.name!="tfoot"):
                trb.append(trbTag[i].contents)
        astTag=boxSoup.findAll("td", {"class": "right ", "data-stat": "ast"})
        for i in range(0,len(astTag)):
            if(astTag[i].parent.parent.name!="tfoot"):
                ast.append(astTag[i].contents)
        stlTag=boxSoup.findAll("td", {"class": "right ", "data-stat": "stl"})
        for i in range(0,len(stlTag)):
            if(stlTag[i].parent.parent.name!="tfoot"):
                stl.append(stlTag[i].contents)    
        blkTag=boxSoup.findAll("td", {"class": "right ", "data-stat": "blk"})
        for i in range(0,len(blkTag)):
            if(blkTag[i].parent.parent.name!="tfoot"):
                blk.append(blkTag[i].contents)
        tovTag=boxSoup.findAll("td", {"class": "right ", "data-stat": "tov"})
        for i in range(0,len(tovTag)):
            if(blkTag[i].parent.parent.name!="tfoot"):
                tov.append(tovTag[i].contents)
        pfTag=boxSoup.findAll("td", {"class": "right ", "data-stat": "pf"})
        for i in range(0,len(pfTag)):
            if(pfTag[i].parent.parent.name!="tfoot"):
                pf.append(pfTag[i].contents)
        ptsTag=boxSoup.findAll("td", {"class": "right ", "data-stat": "pts"})
        for i in range(0,len(ptsTag)):
            if(ptsTag[i].parent.parent.name!="tfoot"):
                pts.append(ptsTag[i].contents)    
        plus_minusTag=boxSoup.findAll("td", {"class": "right ", "data-stat": "plus_minus"})
        for i in range(0,len(plus_minusTag)):
            if(plus_minusTag[i].parent.parent.name!="tfoot"):
                plus_minus.append(plus_minusTag[i].contents)
        ts_pctTag=boxSoup.findAll("td", {"class": "right ", "data-stat": "ts_pct"})
        for i in range(0,len(ts_pctTag)):
            if(ts_pctTag[i].parent.parent.name!="tfoot"):
                ts_pct.append(ts_pctTag[i].contents)
        efg_pctTag=boxSoup.findAll("td", {"class": "right ", "data-stat": "efg_pct"})
        for i in range(0,len(efg_pctTag)):
            if(efg_pctTag[i].parent.parent.name!="tfoot"):
                efg_pct.append(efg_pctTag[i].contents)
        off_rtgTag=boxSoup.findAll("td", {"class": "right ", "data-stat": "off_rtg"})
        for i in range(0,len(off_rtgTag)):
            if(off_rtgTag[i].parent.parent.name!="tfoot"):
                off_rtg.append(off_rtgTag[i].contents)
        def_rtgTag=boxSoup.findAll("td", {"class": "right ", "data-stat": "def_rtg"})
        for i in range(0,len(def_rtgTag)):
            if(def_rtgTag[i].parent.parent.name!="tfoot"):
                def_rtg.append(def_rtgTag[i].contents)
        minutes=[]
    labels=["Team1", "Team2", "Date", "PlayerID", "Minutes", "FG", "FGA", "FGPCT", "3PT", "3PTA", "3PTPCT", "FT", "FTA",
            "FTPCT", "ORB", "DRB", "TRB", "AST", "STL", "BLK", "TOV", "PF", "PTS", "+/-", "TSPCT", "EFGPCT", "OFFRTG", "DEFRTG"]
    boxScore_df=pd.DataFrame({"Team1": team1,
                              "Team2": team2,
                              "Date": date,
                              "PlayerID": playerID,
                              "Minutes": finalMinutes,
                              "FG": fg,
                              "FGA": fga,
                              "FGPCT":fgpct,
                              "3PT": fg3,
                              "3PTA": fg3a,
                              "3PTPCT": fg3pct,
                              "FT": ft,
                              "FTA": fta,
                              "FTPCT": ftpct,
                              "ORB": orb,
                              "DRB": drb,
                              "TRB": trb,
                              "AST": ast,
                              "STL": stl,
                              "BLK": blk,
                              "TOV": tov,
                              "PF": pf,
                              "PTS": pts,
                              "+/-": plus_minus,
                              "TSPCT":ts_pct,
                              "EFGPCT":efg_pct,
                              "OFFRTG": off_rtg,
                              "DEFRTG": def_rtg
        })
    boxScore_df = boxScore_df[labels]


    return boxScore_df

In [430]:
boxScore_df=gameBoxScore(urlBox)

139
139
139
139
139
139


In [432]:
boxScore_df.to_csv("BoxScore02072009.csv")